In [1]:
from pyspark.sql.types import *
import json

schema_json = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile').schema.json()

new_schema = StructType.fromJson(json.loads(schema_json))

In [2]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import udf

streamed_df = spark.readStream.schema(new_schema).json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata')

filtered_df = streamed_df.filter(streamed_df.atStop == True)

def get_timestamp(s):
    return int(s['$numberLong'][:-3])

get_timestamp_udf = udf(get_timestamp,LongType())
filtered_df = filtered_df.withColumn('date', get_timestamp_udf('timestamp').cast('timestamp'))
filtered_df = filtered_df.filter(filtered_df.date > lit('2018-08-01')) #save last two months of the data for testing 

In [3]:
query = (filtered_df
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only)
    .queryName("counts")     # counts = name of the in-memory table
    .start())

In [4]:
# query.stop()

In [5]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)
df = sqlContext.sql("select * from counts")

In [6]:
df.count()

Out[11]: 365973

In [7]:
display(df)

_id,actualDelay,angle,anomaly,areaId,areaId1,areaId2,areaId3,atStop,busStop,calendar,congestion,currentHour,dateType,dateTypeEnum,delay,direction,distanceCovered,ellapsedTime,filteredActualDelay,gridID,journeyPatternId,justLeftStop,justStopped,latitude,lineId,loc,longitude,poiId,poiId2,probability,systemTimestamp,timestamp,vehicleId,vehicleSpeed,date
List(5b963433f0e26f0ccd7e7a1d),0,0.0,false,1011,15,252,1011,true,7574,List(1536557843000),false,5,0,WEEKDAY,0,0,0.0,0,0,"200,12",01451009,false,false,53.182533,145,"List(List(-6.130133, 53.182533), Point)",-6.130133,0,0,0.0,7.6794457E7,List(1536557843000),44224,0,2018-09-10T05:37:23.000+0000
List(5b963434f0e26f0ccd7e7a1e),0,0.0,false,4442,17,277,4442,true,1696,List(1536557843000),false,5,0,WEEKDAY,248,0,0.0,0,0,"133,117",00371001,false,false,53.371435,37,"List(List(-6.329842, 53.371435), Point)",-6.329842,0,0,0.0,7.6795986E7,List(1536557843000),44309,0,2018-09-10T05:37:23.000+0000
List(5b963434f0e26f0ccd7e7a20),0,0.0,false,4241,16,264,4241,true,3227,List(1536557843000),false,5,0,WEEKDAY,151,0,0.0,0,0,"197,56",007A1003,false,false,53.261916,7,"List(List(-6.137744, 53.261916), Point)",-6.137744,0,0,0.0,7.6796082E7,List(1536557843000),38035,0,2018-09-10T05:37:23.000+0000
List(5b963434f0e26f0ccd7e7a21),0,0.0,false,17759,17,277,4439,true,1989,List(1536557843000),false,5,0,WEEKDAY,247,0,0.0,0,0,"137,99",00401004,false,false,53.339599,40,"List(List(-6.319035, 53.339599), Point)",-6.319035,0,0,0.0,7.6796101E7,List(1536557843000),43122,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a22),0,0.0,false,1011,15,252,1011,true,4260,List(1536557843000),false,5,0,WEEKDAY,0,0,0.0,0,0,"197,9",084X1004,false,false,53.177683,145,"List(List(-6.13775, 53.177683), Point)",-6.13775,0,0,0.0,7.6796109E7,List(1536557843000),43017,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a24),0,0.0,false,15169,14,236,3792,true,1167,List(1536557843000),false,5,0,WEEKDAY,-194,0,0.0,0,0,"153,86",00151003,false,false,53.315919,15,"List(List(-6.27138, 53.315919), Point)",-6.27138,0,0,0.0,7.6797014E7,List(1536557843000),44360,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a25),0,0.0,false,4452,17,277,4452,true,73,List(1536557843000),false,5,0,WEEKDAY,131,0,0.0,0,0,"143,115",01220003,false,false,53.367326,122,"List(List(-6.300706, 53.367326), Point)",-6.300706,0,0,0.0,7.6797019E7,List(1536557843000),40004,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a27),0,0.0,false,73085,17,285,4567,true,7391,List(1536557843000),false,5,0,WEEKDAY,0,0,0.0,0,0,"159,99",00660003,false,false,53.339383,26,"List(List(-6.251033, 53.339383), Point)",-6.251033,0,0,0.0,7.6797028E7,List(1536557843000),44011,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a29),0,0.0,false,73068,17,285,4566,true,265,List(1536557843000),false,5,0,WEEKDAY,0,0,0.0,0,0,"156,107",00440006,false,false,53.353433,44,"List(List(-6.261967, 53.353433), Point)",-6.261967,0,0,0.0,7.6797037E7,List(1536557843000),44387,0,2018-09-10T05:37:23.000+0000
List(5b963435f0e26f0ccd7e7a2e),0,0.0,false,563,8,140,563,true,4564,List(1536557844000),false,5,0,WEEKDAY,0,0,0.0,0,0,"107,81",00691006,false,false,53.305733,68,"List(List(-6.409083, 53.305733), Point)",-6.409083,0,0,0.0,7.6797171E7,List(1536557844000),44427,0,2018-09-10T05:37:24.000+0000


In [9]:
df = df[['_id','atStop','busStop','congestion','delay','justLeftStop','justStopped','latitude' ,'longitude','vehicleId','vehicleSpeed','actualDelay','timestamp','journeyPatternId']]

In [10]:
#correct original data and add new fields
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType,TimestampType,StringType,DateType

def get_weekday(s):
  return s.weekday()

def get_hour(s):
  return s.hour

def get_is_weekend(s):
  return 1 if s in [6,7] else 0

def get_id(s):
  return s['$oid']

def get_month(s):
  return s.month

def get_only_date(s):
  return str(s.date())

def get_just_left(s):
  return 1 if s else 0

def get_just_stopped(s):
  return 1 if s else 0

def get_congestion(s):
  return 1 if s else 0

def get_timestamp(s):
    return int(s['$numberLong'][:-3])

get_timestamp_udf = udf(get_timestamp,LongType())
get_weekday_udf = udf(get_weekday, LongType())
get_hour_udf = udf(get_hour, LongType())
get_is_weekend_udf = udf(get_is_weekend, LongType())
get_id_udf = udf(get_id, StringType())
get_month_udf = udf(get_month, LongType())
get_only_date_udf = udf(get_only_date, StringType())
get_just_left_udf = udf(get_just_left, LongType())
get_just_stopped_udf = udf(get_just_stopped, LongType())
get_congestion_udf = udf(get_congestion, LongType())


df = df.withColumn('date', get_timestamp_udf('timestamp').cast('timestamp'))
df = df.withColumn('weekday', get_weekday_udf('date'))
df = df.withColumn('month', get_month_udf('date'))
df = df.withColumn('only_date', get_only_date_udf('date'))
df = df.withColumn('is_weekend', get_is_weekend_udf('weekday'))
df = df.withColumn('hour', get_hour_udf('date'))
df = df.withColumn('id', get_id_udf('_id'))
df = df.withColumn('just_left', get_just_left_udf('justLeftStop'))
df = df.withColumn('just_stopped', get_just_stopped_udf('justStopped'))
df = df.withColumn('conges', get_just_left_udf('congestion'))
df = df.drop(*['_id','atStop','justLeftStop','justStopped','congestion','timestamp'])

In [11]:
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import lit

#keep only transitions between stops
my_window = Window.partitionBy('vehicleId').orderBy('date')
target_df = df.withColumn('origin', f.lag(df.busStop.cast("bigint")).over(my_window))
target_df = target_df[target_df['busStop'] != target_df['origin']]

#calculate time difference between stops
target_df = target_df.withColumn('prev_ts', f.lag(target_df.date.cast("bigint")).over(my_window))
target_df = target_df.withColumn('time_to_reach_next', f.when(f.isnull(target_df.date.cast("bigint") - target_df.prev_ts), 0)
                            .otherwise(target_df.date.cast("bigint") - target_df.prev_ts))
target_df = target_df[ target_df['time_to_reach_next'] != 0]

#transform into training data
target_df = target_df.withColumnRenamed('busStop', 'dest')
target_df = target_df.drop('prev_ts')

#add field of from->to
def get_from_to(frm,to):
  return str(frm)+'->'+str(to)

get_from_to_udf = udf(get_from_to, StringType())
target_df = target_df.withColumn('from_to', lit(get_from_to_udf(f.col('origin'),f.col('dest'))))

w = Window.partitionBy('from_to')
target_df = target_df.withColumn('seg_count', f.count('from_to').over(w))

In [12]:
target_df = target_df[target_df.time_to_reach_next < 7200] #drop rows with more then two hours between stops
target_df = target_df[target_df.time_to_reach_next > 30]  #drop rows with less then 30 seconds between stops
target_df = target_df.filter(target_df.seg_count > 20)

In [13]:
target_df.count()

Out[28]: 96741

In [14]:
display(target_df)

dest,delay,latitude,longitude,vehicleId,vehicleSpeed,actualDelay,journeyPatternId,date,weekday,month,only_date,is_weekend,hour,id,just_left,just_stopped,conges,origin,time_to_reach_next,from_to,seg_count
1170,100,53.321637,-6.266519,43110,11,0,00151003,2018-09-10T15:45:55.000+0000,0,9,2018-09-10,0,15,5b9691bbf0e26f3d4e2d3b05,0,1,0,1166,195,1166->1170,40
1170,63,53.321637,-6.266519,33491,10,0,015B1005,2018-09-11T11:08:20.000+0000,1,9,2018-09-11,0,11,5b97a228f0e26f5879313376,0,1,0,1166,161,1166->1170,40
1170,130,53.321637,-6.266519,44033,29,0,00141002,2018-09-11T14:48:28.000+0000,1,9,2018-09-11,0,14,5b97d5c1f0e26f587932feb5,0,1,0,1166,74,1166->1170,40
1170,422,53.321637,-6.266519,38004,0,0,00651003,2018-09-11T08:08:32.000+0000,1,9,2018-09-11,0,8,5b977807f0e26f3d4e376a2c,0,1,0,1166,141,1166->1170,40
1170,187,53.321637,-6.266519,38062,10,0,015A1003,2018-09-11T15:50:18.000+0000,1,9,2018-09-11,0,15,5b97e43ff0e26f5879347a0e,0,1,0,1166,134,1166->1170,40
1170,1330,53.321637,-6.266519,33480,9,0,015A1003,2018-09-11T07:34:30.000+0000,1,9,2018-09-11,0,7,5b97700bf0e26f3d4e3666cd,0,1,0,1166,279,1166->1170,40
1170,-423,53.321637,-6.266519,44111,27,0,00141002,2018-09-10T09:58:13.000+0000,0,9,2018-09-10,0,9,5b964040f0e26f3d4e28f889,0,1,0,1166,120,1166->1170,40
1170,184,53.321637,-6.266519,33479,2,19,015A1003,2018-09-10T14:50:07.000+0000,0,9,2018-09-10,0,14,5b9684a5f0e26f3d4e2bffbe,0,1,0,1166,150,1166->1170,40
1170,-49,53.321637,-6.266519,33485,10,0,015A1003,2018-09-11T14:47:02.000+0000,1,9,2018-09-11,0,14,5b97d56af0e26f587932f7ab,0,1,0,1166,147,1166->1170,40
1170,599,53.321637,-6.266519,44091,2,17,015B1005,2018-09-11T06:59:50.000+0000,1,9,2018-09-11,0,6,5b9767ebf0e26f3d4e3549e8,0,1,0,1166,301,1166->1170,40


In [15]:
learn_df = target_df.drop(*['dest','time_to_reach_next','from_to','seg_count'])

In [16]:
true_paths = spark.read.options(header='true', inferSchema = 'true').load('/user/hive/warehouse/true_paths_v2')
true_paths = true_paths.withColumnRenamed('origin', 'tp_origin')
true_paths = true_paths.withColumnRenamed('journeyPatternId', 'tp_journeyPatternId')
display(true_paths)

tp_journeyPatternId,tp_origin,dest,top_score
00010001,229,227,0.5172670807453417
00010003,223,226,0.7777777777777778
00040001,1359,324,1.0
00040001,2142,324,1.0
00041003,3084,469,0.5494186046511628
00041003,4725,7330,0.6666666666666666
00070006,2036,2040,0.8871181938911022
00070009,3215,3217,0.38697318007662834
00071005,421,7639,1.0
00090001,6094,7132,1.0


In [17]:
learn_df = learn_df.join(true_paths,(learn_df.journeyPatternId == true_paths.tp_journeyPatternId) & (learn_df.origin == true_paths.tp_origin), how = 'left')

In [18]:
learn_df = learn_df.drop(*['top_score','tp_origin','tp_journeyPatternId'])

In [19]:
#add field of from->to
learn_df = learn_df.withColumn('from_to', lit(get_from_to_udf(f.col('origin'),f.col('dest'))))

In [20]:
display(learn_df)

delay,latitude,longitude,vehicleId,vehicleSpeed,actualDelay,journeyPatternId,date,weekday,month,only_date,is_weekend,hour,id,just_left,just_stopped,conges,origin,journeyPatternId,dest,from_to
100,53.321637,-6.266519,43110,11,0,00151003,2018-09-10T15:45:55.000+0000,0,9,2018-09-10,0,15,5b9691bbf0e26f3d4e2d3b05,0,1,0,1166,00151003,1167,1166->1167
-395,53.321637,-6.266519,43073,13,0,015B1005,2018-09-10T14:20:47.000+0000,0,9,2018-09-10,0,14,5b967dc2f0e26f3d4e2b5e29,0,1,0,1166,015B1005,1167,1166->1167
-140,53.321637,-6.266519,44194,17,0,00141002,2018-09-10T10:43:06.000+0000,0,9,2018-09-10,0,10,5b964ac2f0e26f3d4e29a6a8,0,1,0,1166,00141002,1167,1166->1167
347,53.321637,-6.266519,44194,10,0,00141002,2018-09-11T07:26:35.000+0000,1,9,2018-09-11,0,7,5b976e34f0e26f3d4e362469,0,1,0,1166,00141002,1167,1166->1167
-131,53.321637,-6.266519,44095,17,0,00151003,2018-09-10T16:54:44.000+0000,0,9,2018-09-10,0,16,5b96a1d7f0e26f3d4e2f2817,0,1,0,1166,00151003,1167,1166->1167
422,53.321637,-6.266519,38004,0,0,00651003,2018-09-11T08:08:32.000+0000,1,9,2018-09-11,0,8,5b977807f0e26f3d4e376a2c,0,1,0,1166,00651003,1167,1166->1167
140,53.321637,-6.266519,33481,3,0,015A1003,2018-09-11T10:51:09.000+0000,1,9,2018-09-11,0,10,5b979e22f0e26f587930dbf5,0,1,0,1166,015A1003,1167,1166->1167
-423,53.321637,-6.266519,44111,27,0,00141002,2018-09-10T09:58:13.000+0000,0,9,2018-09-10,0,9,5b964040f0e26f3d4e28f889,0,1,0,1166,00141002,1167,1166->1167
58,53.321637,-6.266519,43080,17,15,00151003,2018-09-11T17:07:53.000+0000,1,9,2018-09-11,0,17,5b97f670f0e26f587936aaac,0,1,0,1166,00151003,1167,1166->1167
293,53.321637,-6.266519,44433,16,0,015A1003,2018-09-10T18:20:16.000+0000,0,9,2018-09-10,0,18,5b96b5ebf0e26f3d4e3144cb,0,1,0,1166,015A1003,1167,1166->1167


In [21]:
checker = target_df[['id','from_to']]
checker = checker.withColumnRenamed('id', '_id')
checker = checker.withColumnRenamed('from_to', 'true_from_to')
checked_df = learn_df.join(checker,(learn_df.id == checker._id) & (learn_df.from_to == checker.true_from_to))
print(checked_df.count() / target_df.count())

0.7873290538654758

In [22]:
segment_data = spark.read.options(header='true', inferSchema = 'true').load('/user/hive/warehouse/segment_data')
segment_data = segment_data.withColumnRenamed('from_to', 'sd_from_to')

In [23]:
display(segment_data)

sd_from_to,mean_seg,stdev_seg,max_in_seg,min_in_seg,prev_time,distnce_between,dis_from_center,speed_in_seg
3227->3228,68.05488685604237,27.834441582286026,150.0,33.0,65.0379684418146,516.0672193166067,15902.053823617789,8.24177000747554
4359->2689,1305.0894308943084,543.0836968392845,2770.0,378.0,1299.1138211382113,2973.301527845181,12332.483565167682,2.872410077564367
2787->2182,105.42857142857146,37.665120684950416,200.0,35.0,102.44797687861272,865.1669916582935,9455.562536691654,9.158172657993507
4337->1317,92.33753340969822,39.53659965363507,209.0,31.0,90.04978269458712,325.12269513911286,5850.8597807113665,3.9464924491591367
1119->1120,107.37506209637357,120.96450489869439,470.0,31.0,93.90399584846912,569.0309920496562,5247.201487598112,8.301747510908346
2655->2673,289.12681912681893,124.50471262862271,634.0,112.0,275.64453961456104,2508.36112476161,11818.656358739294,9.922442128677847
3669->1637,481.55492424242436,89.36169018327328,737.0,279.0,478.97714285714284,2821.9681955550964,8415.414118565152,6.0675912987782565
6310->7398,1221.6079999999988,297.5225634165172,1928.0,843.0,1178.9344262295083,11146.426305551999,6791.1455788534195,9.610230559197602
1403->1404,53.66405638214568,31.00140152797957,145.0,31.0,50.52955465587045,355.76558571016255,3981.2482180699644,7.510515760722916
2319->1099,393.34426229508216,95.45478532366423,662.0,260.0,390.5983606557377,2480.4839127337345,7776.716168513063,6.641263843620588


In [24]:
df_to_predict = checked_df.drop(*['_id','true_from_to'])
df_to_predict = df_to_predict.join(segment_data, df_to_predict.from_to == segment_data.sd_from_to)

In [25]:
display(df_to_predict)

delay,latitude,longitude,vehicleId,vehicleSpeed,actualDelay,journeyPatternId,date,weekday,month,only_date,is_weekend,hour,id,just_left,just_stopped,conges,origin,dest,from_to,sd_from_to,mean_seg,stdev_seg,max_in_seg,min_in_seg,prev_time,distnce_between,dis_from_center,speed_in_seg
-139,53.347977,-6.292855,44132,14,0,00670001,2018-09-11T05:38:04.000+0000,1,9,2018-09-11,0,5,5b9754c3f0e26f3d4e3373f5,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
25,53.347977,-6.292855,44222,0,0,00660003,2018-09-10T15:39:07.000+0000,0,9,2018-09-10,0,15,5b96901ff0e26f3d4e2d102d,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
140,53.347977,-6.292855,33566,7,0,00670001,2018-09-11T06:42:50.000+0000,1,9,2018-09-11,0,6,5b9763eff0e26f3d4e34c8a5,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
0,53.347977,-6.292855,44294,14,0,00670001,2018-09-10T16:16:11.000+0000,0,9,2018-09-10,0,16,5b9698d0f0e26f3d4e2e10ac,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
866,53.347977,-6.292855,33596,17,0,00260002,2018-09-11T15:39:46.000+0000,1,9,2018-09-11,0,15,5b97e1c9f0e26f5879343616,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
761,53.347977,-6.292855,33596,17,21,00660003,2018-09-11T17:20:26.000+0000,1,9,2018-09-11,0,17,5b97f95ff0e26f58793703a4,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
-210,53.347977,-6.292855,33432,7,18,00660003,2018-09-10T14:17:54.000+0000,0,9,2018-09-10,0,14,5b967d1bf0e26f3d4e2b4de9,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
399,53.347977,-6.292855,44415,0,16,00670001,2018-09-11T16:23:06.000+0000,1,9,2018-09-11,0,16,5b97ebf3f0e26f58793563d2,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
151,53.347977,-6.292855,44012,0,20,00260002,2018-09-10T11:27:11.000+0000,0,9,2018-09-10,0,11,5b96551af0e26f3d4e2a7fed,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482
-57,53.347977,-6.292855,44348,7,0,00690004,2018-09-11T08:13:04.000+0000,1,9,2018-09-11,0,8,5b977916f0e26f3d4e378a8f,0,1,0,1445,7078,1445->7078,1445->7078,225.75808402548157,45.37308980434621,361.0,100.0,224.4281069657807,1680.7907541903367,3376.84805324889,7.77705946010482


In [26]:
original_data = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile')

In [27]:
#filter data
total_df = original_data[original_data.atStop == True]
total_df = total_df[['_id','atStop','busStop','congestion','delay','justLeftStop','justStopped','latitude' ,'longitude','vehicleId','vehicleSpeed','actualDelay','timestamp','journeyPatternId']]

#correct attributes
total_df = total_df.withColumn('date', get_timestamp_udf('timestamp').cast('timestamp'))
total_df = total_df.withColumn('weekday', get_weekday_udf('date'))
total_df = total_df.withColumn('month', get_month_udf('date'))
total_df = total_df.withColumn('only_date', get_only_date_udf('date'))
total_df = total_df.withColumn('is_weekend', get_is_weekend_udf('weekday'))
total_df = total_df.withColumn('hour', get_hour_udf('date'))
total_df = total_df.withColumn('id', get_id_udf('_id'))
total_df = total_df.withColumn('just_left', get_just_left_udf('justLeftStop'))
total_df = total_df.withColumn('just_stopped', get_just_stopped_udf('justStopped'))
total_df = total_df.withColumn('conges', get_just_left_udf('congestion'))
total_df = total_df.drop(*['_id','atStop','justLeftStop','justStopped','congestion','timestamp'])

#keep only transitions between stops
my_window = Window.partitionBy('vehicleId').orderBy('date')
learn_df = total_df.withColumn('origin', f.lag(total_df.busStop.cast("bigint")).over(my_window))
learn_df = learn_df[learn_df['busStop'] != learn_df['origin']]

#calculate time difference between stops
learn_df = learn_df.withColumn('prev_ts', f.lag(learn_df.date.cast("bigint")).over(my_window))
learn_df = learn_df.withColumn('time_to_reach_next', f.when(f.isnull(learn_df.date.cast("bigint") - learn_df.prev_ts), 0)
                            .otherwise(learn_df.date.cast("bigint") - learn_df.prev_ts))
learn_df = learn_df[ learn_df['time_to_reach_next'] != 0]

#transform into training data
learn_df = learn_df.withColumnRenamed('busStop', 'dest')
learn_df = learn_df.drop('prev_ts')

#add field of from->to
get_from_to_udf = udf(get_from_to, StringType())
learn_df = learn_df.withColumn('from_to', lit(get_from_to_udf(f.col('origin'),f.col('dest'))))

w = Window.partitionBy('from_to')
learn_df = learn_df.withColumn('seg_count', f.count('from_to').over(w))

learn_df = learn_df[learn_df.time_to_reach_next < 7200] #drop rows with more then two hours between stops
learn_df = learn_df[learn_df.time_to_reach_next > 30]  #drop rows with less then 30 seconds between stops
learn_df = learn_df.filter(learn_df.seg_count > 100) #take only comoon segments 
learn_df = learn_df.filter(learn_df.date < lit('2018-08-01')) #save last two months of the data for testing in streaming (August\July 2018)

my_window = Window.partitionBy('from_to')
#mean time segment
learn_df = learn_df.withColumn('mean_seg', f.mean('time_to_reach_next').over(my_window))
#variance in segment
learn_df = learn_df.withColumn('stdev_seg', f.stddev('time_to_reach_next').over(my_window))

#remove outlayres, greater than 3 stdevs from avg or or less than 2 stdevs 
learn_df = learn_df[learn_df.time_to_reach_next < (learn_df.mean_seg + 3 * learn_df.stdev_seg)]
learn_df = learn_df[learn_df.time_to_reach_next > (learn_df.mean_seg - 3 * learn_df.stdev_seg)]

#max time segment
learn_df = learn_df.withColumn('max_in_seg', f.max('time_to_reach_next').over(my_window))

#min time segment
learn_df = learn_df.withColumn('min_in_seg', f.min('time_to_reach_next').over(my_window))

#prev_time_in_segment
my_window = Window.partitionBy('from_to').orderBy('date')
learn_df = learn_df.withColumn('prev_time', f.lag(learn_df.time_to_reach_next).over(my_window))

#calculate distances
my_window = Window.partitionBy('vehicleId').orderBy('date')
learn_df = learn_df.withColumn('prev_lat', f.lag(learn_df.latitude).over(my_window))
learn_df = learn_df.withColumn('prev_lon', f.lag(learn_df.longitude).over(my_window))   
learn_df = learn_df.na.fill(0)
learn_df = learn_df[learn_df['prev_lat'] != 0]

from math import radians, cos, sin, asin, sqrt

@f.udf("float")
def get_distance(longit_a, latit_a, longit_b, latit_b):
    if None in [longit_a, latit_a, longit_b, latit_b]:
        return 9999
    # Transform to radians
    longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a,  latit_a, longit_b, latit_b])
    dist_longit = longit_b - longit_a
    dist_latit = latit_b - latit_a
    # Calculate area
    area = sin(dist_latit/2)**2 + cos(latit_a) * cos(latit_b) * sin(dist_longit/2)**2
    # Calculate the central angle
    central_angle = 2 * asin(sqrt(area))
    radius = 6371 # THIS IN KM
    # Calculate Distance
    distance = central_angle * radius
    return distance * 1000
  
@f.udf("float")
def get_dis_from_center(longit_a, latit_a):
  longit_b, latit_b = 53.3422665, -6.2554468 #city center coordinates
  if None in [longit_a, latit_a, longit_b, latit_b]:
      return 9999
  # Transform to radians
  longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a,  latit_a, longit_b, latit_b])
  dist_longit = longit_b - longit_a
  dist_latit = latit_b - latit_a
  # Calculate area
  area = sin(dist_latit/2)**2 + cos(latit_a) * cos(latit_b) * sin(dist_longit/2)**2
  # Calculate the central angle
  central_angle = 2 * asin(sqrt(area))
  radius = 6371 # THIS IN KM
  # Calculate Distance
  distance = central_angle * radius
  return distance * 1000

learn_df = learn_df.withColumn('distnce_between',get_distance(f.col('prev_lat'),f.col('prev_lon'),f.col('latitude'),f.col('longitude')))
learn_df = learn_df.withColumn('lat_between',(learn_df.prev_lat + learn_df.latitude)/ 2)
learn_df = learn_df.withColumn('lon_between',(learn_df.prev_lon + learn_df.longitude)/ 2)
learn_df = learn_df.withColumn('dis_from_center',get_dis_from_center(f.col('lat_between'),f.col('lon_between')))
learn_df = learn_df[learn_df['distnce_between'] < 100000]
learn_df = learn_df[learn_df['dis_from_center'] < 100000]
learn_df = learn_df.withColumn('speed_in_seg',learn_df.distnce_between / learn_df.time_to_reach_next)
learn_df = learn_df[learn_df['speed_in_seg'] < 55] #drop segments with speed over 200kmh
learn_df = learn_df.drop('prev_lat','prev_lon','lat_between','lon_between')

In [28]:
small_list = learn_df.take(1000)
small_df = spark.createDataFrame(small_list, learn_df.columns)

In [29]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
import numpy 

numericCols =['delay','vehicleSpeed','actualDelay','max_in_seg','min_in_seg','stdev_seg','distnce_between','dis_from_center','hour','latitude','longitude','mean_seg','prev_time','speed_in_seg']
assembler = VectorAssembler(inputCols = numericCols, outputCol="features")
ml_df = assembler.transform(learn_df)

# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(ml_df)

# Make predictions
predictions = model.transform(ml_df)

# Shows the result.
centers = model.clusterCenters()
centroids = {}
for i,center in enumerate(centers):
  centroids[i] = [center, center[-1]]

In [30]:
helper_df = df_to_predict.toPandas()

def impute_speed(a,b,c,d,e,f,g,h,i,j,k,l,m,n):
  fetur_vec = numpy.asarray([a,b,c,d,e,f,g,h,i,j,k,l,m,n])
  speed = 0
  fuzzifier = 1.3
  distances_from_centroids = {}
  for k,val in centroids.items(): #calculate distances for all centroids and save speed in each one
    distances_from_centroids[k] = [numpy.linalg.norm(fetur_vec - val[0]) ** (-2/fuzzifier - 1) , val[1]] # calculate distance from centroids, save speed
  total_dis = 0 
  for v in distances_from_centroids.values():
    total_dis += v[0]
    
  for k,val in distances_from_centroids.items():
    speed += (val[0] * val[1] ) / total_dis #add speed times distance from the centroids
  return speed

helper_df['new_speed'] = [impute_speed(a,b,c,d,e,f,g,h,i,j,k,l,m,n) for a,b,c,d,e,f,g,h,i,j,k,l,m,n in zip(helper_df.delay, helper_df.vehicleSpeed, helper_df.actualDelay, helper_df.max_in_seg, helper_df.min_in_seg, helper_df.stdev_seg, helper_df.distnce_between, helper_df.dis_from_center, helper_df.hour, helper_df.latitude, helper_df.longitude, helper_df.mean_seg, helper_df.prev_time, helper_df.speed_in_seg)]


In [31]:
sqlCtx = SQLContext(sc)
altered_df = sqlCtx.createDataFrame(helper_df)

In [32]:
altered_df = altered_df.drop('speed_in_seg')
altered_df = altered_df.withColumnRenamed('new_speed','speed_in_seg')

In [33]:
from pyspark.ml.regression import GBTRegressionModel
streaming_gbt_model = GBTRegressionModel.load('/user/hive/warehouse/streaming_gbt_model')

In [34]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

altered_df = altered_df.withColumn('time_to_reach_next', lit(0)) #add field to be predicted

ml_df = altered_df

numericCols =['delay','vehicleSpeed','actualDelay','max_in_seg','min_in_seg','stdev_seg','distnce_between','dis_from_center','hour','speed_in_seg','latitude','longitude','mean_seg','prev_time']
assembler = VectorAssembler(inputCols = numericCols, outputCol="features")
ml_df = assembler.transform(ml_df)
ml_df = ml_df.select([ 'features','id'])

In [35]:
gbt_predictions = streaming_gbt_model.transform(ml_df)
preds = gbt_predictions.select("prediction","id")
preds = preds.withColumnRenamed('id','_id')

In [36]:
predicted_df = altered_df.join(preds, altered_df.id == preds._id)
predicted_df = predicted_df.drop(*['time_to_reach_next','_id'])
predicted_df = predicted_df.withColumnRenamed('prediction','time_to_reach_next')

In [37]:
# predicted_df.write.saveAsTable("predicted_df")
predicted_df = spark.read.options(header='true', inferSchema = 'true').load('/user/hive/warehouse/predicted_df')

In [38]:
display(predicted_df[['journeyPatternId','from_to','mean_seg','speed_in_seg','time_to_reach_next']])

journeyPatternId,from_to,mean_seg,speed_in_seg,time_to_reach_next
01231001,4518->509,85.41085899513774,5.186134152703045,88.92921419909192
01400003,6229->6364,57.34652383034542,7.9506655769456005,46.12935945464972
00071008,2041->3071,117.68272023233308,7.752837870332268,90.01042876863866
00270004,6130->1262,66.27543659832953,7.944515554296605,70.73132424569441
00070008,416->417,69.3415795586528,5.271007489573638,72.31117903208228
00271001,2379->2380,60.617735303922686,5.209307360653201,60.295493065727065
00070008,423->424,80.28022471910111,7.608975140351362,66.09665904741827
039A1001,1863->1864,48.69296577946769,7.750656394019913,44.046302711388044
00701001,1479->315,160.60432925931352,5.212914751699954,145.39734879163277
00150003,4415->301,244.29989090115606,5.245932866519469,223.242172820139


In [39]:
display(predicted_df[['journeyPatternId','from_to','mean_seg','speed_in_seg','time_to_reach_next']])

journeyPatternId,from_to,mean_seg,speed_in_seg,time_to_reach_next
01231001,4518->509,85.41085899513774,5.186134152703045,88.92921419909192
01400003,6229->6364,57.34652383034542,7.9506655769456005,46.12935945464972
00071008,2041->3071,117.68272023233308,7.752837870332268,90.01042876863866
00270004,6130->1262,66.27543659832953,7.944515554296605,70.73132424569441
00070008,416->417,69.3415795586528,5.271007489573638,72.31117903208228
00271001,2379->2380,60.617735303922686,5.209307360653201,60.295493065727065
00070008,423->424,80.28022471910111,7.608975140351362,66.09665904741827
039A1001,1863->1864,48.69296577946769,7.750656394019913,44.046302711388044
00701001,1479->315,160.60432925931352,5.212914751699954,145.39734879163277
00150003,4415->301,244.29989090115606,5.245932866519469,223.242172820139


In [40]:
coded_df = predicted_df.withColumn('country_code', lit('IRL'))
display(coded_df[['country_code','latitude','longitude']])

country_code,latitude,longitude
IRL,53.357024,-6.264297
IRL,53.371689,-6.253159
IRL,53.34597,-6.25614
IRL,53.300767,-6.28395
IRL,53.388853,-6.196527
IRL,53.1524,-6.081623
IRL,53.428052,-6.244154
IRL,53.390714,-6.429783
IRL,53.349386,-6.252166
IRL,53.401698,-6.181091


In [41]:
true_results = target_df[['id','time_to_reach_next']]
true_results = true_results.withColumnRenamed('id','_id')
true_results = true_results.withColumnRenamed('time_to_reach_next','true_time_to_reach_next')
final_df = predicted_df.join(true_results, predicted_df.id == true_results._id)
final_df = final_df.withColumn('squared_error', (final_df.time_to_reach_next - final_df.true_time_to_reach_next)** 2)

In [42]:
mean_time = 0
for x in final_df.select('true_time_to_reach_next').rdd.flatMap(lambda x: x).collect():
  mean_time += x
mean_time = mean_time / len(final_df.select('true_time_to_reach_next').rdd.flatMap(lambda x: x).collect())
filtered_df = final_df.filter(final_df.true_time_to_reach_next < 1200)

In [43]:
total_se = 0
for x in filtered_df.select('squared_error').rdd.flatMap(lambda x: x).collect():
  total_se += x
RMSE = (total_se / len(filtered_df.select('squared_error').rdd.flatMap(lambda x: x).collect())) ** (1/2)

In [44]:
print(RMSE)

94.51673335766984

In [45]:
display(final_df[['journeyPatternId','from_to','mean_seg','time_to_reach_next','true_time_to_reach_next','squared_error']])

journeyPatternId,from_to,mean_seg,time_to_reach_next,true_time_to_reach_next,squared_error
00660003,1445->7078,225.75808402548157,264.68355470464957,200,4183.962249229393
00250002,1445->7078,225.75808402548157,266.2783025563138,253,176.3133187770097
066A0002,1445->7078,225.75808402548157,266.2783025563138,300,1137.152878483512
00260002,1445->7078,225.75808402548157,269.67506502090555,220,2467.612084831194
00670001,1445->7078,225.75808402548157,264.68355470464957,260,21.935684671445085
00670001,1445->7078,225.75808402548157,269.67506502090555,261,75.256753116939
00670001,1445->7078,225.75808402548157,264.68355470464957,238,712.012091676026
00690004,1445->7078,225.75808402548157,264.68355470464957,241,560.9107634481286
00690004,1445->7078,225.75808402548157,264.68355470464957,246,349.07521640163293
00670001,1445->7078,225.75808402548157,264.68355470464957,201,4055.595139820094
